In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
pwd

'/content'

In [4]:
cd 'drive/My Drive/upwork/Face_recognition-master'

/content/drive/My Drive/upwork/Face_recognition-master


In [5]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 88kB/s 
     |████████████████████████████████| 3.2MB 35.9MB/s 
     |████████████████████████████████| 491kB 58.6MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [6]:
!pip install tqdm

In [7]:
!pip install scikit-learn==0.20.3

     |████████████████████████████████| 5.4MB 2.8MB/s 
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3


In [8]:
!pip install scikit-image==0.14.2

     |████████████████████████████████| 25.3MB 94kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scikit-image 0.15.0
    Uninstalling scikit-image-0.15.0:
      Successfully uninstalled scikit-image-0.15.0


In [9]:
!pip install mtcnn==0.0.9

     |████████████████████████████████| 2.3MB 2.8MB/s 
  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp36-none-any.whl size=2257690 sha256=d50c98eb723400c95ae43fdc7bf40e6cc1a5448389cc8fa039971249193c79b2
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
Successfully built mtcnn


In [0]:
base = ''

In [11]:
from imgaug import augmenters as iaa
from Manager import *
from VisionUtils import *
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
import imgaug as ia
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
tf.enable_eager_execution()

In [15]:
IMG_RT_PATH = 'data/images/'
FEATURE_RT_PATH = "data/processed_2/features"
FEATURE_AUG_RT_PATH = "data/processed_2/motion_blur_features/"

#img_list = pd.read_csv(IMG_RT_PATH + "test_list.TXT")
feature_extractor = load_model("models/recognition/model/saved_model.h5")
face_detector = MTCNN()






Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.



In [0]:
seq = iaa.Sequential([iaa.MotionBlur(k = 20)])

In [0]:
os.listdir("data/processed_2/")

['motion_blur_features', 'features', 'packaged']

In [0]:
# import os 
# img_path_list = []
# folder_list = os.listdir(IMG_RT_PATH)
# for FOLDER in tqdm(folder_list):
#     if '.DS_Store' in FOLDER:
#       continue
#     image_list_names = os.listdir(IMG_RT_PATH + FOLDER)
#     for NAME in image_list_names:
#         img_path_list.append(FOLDER+'/'+NAME)

In [0]:
# img_path_list[:3]

In [0]:
FEATURE_RT_PATH = "data/processed_2/features/"

In [0]:
feat_path_list = []
folder_list = os.listdir(FEATURE_RT_PATH)
for FOLDER in tqdm(folder_list):
    image_list_names = os.listdir(FEATURE_RT_PATH + FOLDER)
    for NAME in image_list_names:
        feat_path_list.append(FOLDER+'/'+NAME)

100%|██████████| 74/74 [00:00<00:00, 622.09it/s]


In [0]:
feat_path_list[0]

'Tom_Holland_Goes_Undercover_on_Reddit,_YouTube_and_Twitter___GQkRK6fAiMQM4_mp4/Tom_Holland_Goes_Undercover_on_Reddit,_YouTube_and_Twitter___GQkRK6fAiMQM4_mp4_0.npy'

In [0]:
folder_list[0]

'Tom_Holland_Goes_Undercover_on_Reddit,_YouTube_and_Twitter___GQkRK6fAiMQM4_mp4'

In [0]:
img_list = pd.DataFrame({"path": feat_path_list})

In [0]:
img_list

,path
0,"Tom_Holland_Goes_Undercover_on_Reddit,_YouTube..."
1,"Tom_Holland_Goes_Undercover_on_Reddit,_YouTube..."
2,"Tom_Holland_Goes_Undercover_on_Reddit,_YouTube..."
3,"Tom_Holland_Goes_Undercover_on_Reddit,_YouTube..."
4,"Tom_Holland_Goes_Undercover_on_Reddit,_YouTube..."
...,...
7052,6_Things_I_Wish_I_Knew_When_I_Started_Programm...
7053,6_Things_I_Wish_I_Knew_When_I_Started_Programm...
7054,6_Things_I_Wish_I_Knew_When_I_Started_Programm...
7055,6_Things_I_Wish_I_Knew_When_I_Started_Programm...


In [0]:
iman = ImageManager(
    imgpaths = img_list,
    feature_extractor = feature_extractor,
    face_detector =  face_detector, 
    IMG_RTPATH = IMG_RT_PATH,
)

In [0]:
iman.balance_random_sample(100)
paths = iman.get_sample_image_paths()
iman.split(train_size = 0.8)

In [0]:
path1 = np.array(img_list)[iman.sample_feat1]
path2 = np.array(img_list)[iman.sample_feat2]
path1, path2 = shuffle(path1, path2, random_state = 42)

In [0]:
len(path1)*0.75

14430.0

In [0]:
path1[:10]

array([["IBM's_Bridget_van_Kralingen__How_I_WorkBKkLZ4xdkSM_mp4/IBM's_Bridget_van_Kralingen__How_I_WorkBKkLZ4xdkSM_mp4_75.npy"],
       ['Anthem_CEO_Joseph_Swedish__How_I_WorkxRgn8PArbyg_mp4/Anthem_CEO_Joseph_Swedish__How_I_WorkxRgn8PArbyg_mp4_110.npy'],
       ["Nikolaj_CosterWaldau_Answers_the_Web's_Most_Searched_Questions___WIREDsYSXyxEh8Sc_mp4/Nikolaj_CosterWaldau_Answers_the_Web's_Most_Searched_Questions___WIREDsYSXyxEh8Sc_mp4_120.npy"],
       ["LinkedIn's_Reid_Hoffman__How_I_WorkKPUqcwORKDY_mp4/LinkedIn's_Reid_Hoffman__How_I_WorkKPUqcwORKDY_mp4_355.npy"],
       ["IBM's_Bridget_van_Kralingen__How_I_WorkBKkLZ4xdkSM_mp4/IBM's_Bridget_van_Kralingen__How_I_WorkBKkLZ4xdkSM_mp4_485.npy"],
       ['How_we_do!!_The_game_&_50_cent,_REACTION_VIDEO!!!4kE4XDPz1A_mp4/How_we_do!!_The_game_&_50_cent,_REACTION_VIDEO!!!4kE4XDPz1A_mp4_145.npy'],
       ['Eminem_Spits_His_Favorite_50_Cent_VerseG6LuDuaJ5Ro_mp4/Eminem_Spits_His_Favorite_50_Cent_VerseG6LuDuaJ5Ro_mp4_390.npy'],
       ['50_Cent_ft_Emi

In [0]:
xtrain1 = path1[:int(len(path1)*.75)]
xtrain2 = path2[:int(len(path2)*.75)]
xtest1 = path1[int(len(path1)*.75):]
xtest2 = path2[int(len(path2)*.75):]

In [0]:
def extract_feature2( feature_extractor, img, bb):
    """
    Extract features given img, bounding box and feature extractor
    """
    insz = feature_extractor.input_shape
    img = resize(img,(insz[1],insz[2]))
    img = normalize(img).reshape((1, insz[1],insz[2], insz[3]))
    image_feature = feature_extractor.predict(img)
    return image_feature

In [0]:
i = 0
augment_mode = False
method = "imgaug"
training_features1 = []
training_features2 = []
p2, p1, train_labels = [], [], []

In [0]:
path1 = xtrain1
path2 = xtrain2

In [0]:
len(path1)

14430

In [0]:
path1[i][0].split('/')[0]

"IBM's_Bridget_van_Kralingen__How_I_WorkBKkLZ4xdkSM_mp4"

In [0]:
FEATURE_RT_PATH + str(path1[i][0])

"data/processed/motion_blur_features/IBM's_Bridget_van_Kralingen__How_I_WorkBKkLZ4xdkSM_mp4/IBM's_Bridget_van_Kralingen__How_I_WorkBKkLZ4xdkSM_mp4_75.npy"

In [0]:
for i in tqdm(range(len(path1))):
    training_features1.append(np.load(FEATURE_RT_PATH + str(path1[i][0])))
    training_features2.append(np.load(FEATURE_RT_PATH + str(path2[i][0])))
    if(path1[i][0].split('/')[0] ==path2[i][0].split('/')[0]):
        train_labels.append(1)
    else:
        train_labels.append(0)  

 46%|████▌     | 6592/14430 [1:05:03<37:21,  3.50it/s]

In [0]:
print("Number of features in feat1: {}".format(len(training_features1)))
print("Number of features in feat2: {}".format(len(training_features2)))
print("Number of features in labels: {}".format(len(train_labels)))

Number of features in feat1: 48465
Number of features in feat2: 48465
Number of features in labels: 48465


In [0]:
len(training_features1)

48465

In [0]:
len(train_labels)

48465

In [0]:
path1_t = xtest1
path2_t = xtest2
test_features1, test_features2 = [], []
test_labels = []

In [0]:
for i in tqdm(range(len(path1_t))):
    test_features1.append(np.load(FEATURE_RT_PATH + path1_t[i][0]))
    test_features2.append(np.load(FEATURE_RT_PATH + path2_t[i][0]))
    if(path1_t[i][0].split('/')[0] == path2_t[i][0].split('/')[0]):
        test_labels.append(1)
    else:
        test_labels.append(0)

100%|██████████████████████████████████████████████████████████████████| 16155/16155 [02:01<00:00, 133.16it/s]


In [0]:
train_feat1 = np.vstack(training_features1)
train_feat2 = np.vstack(training_features2)

test_feat1 = np.vstack(test_features1)
test_feat2 = np.vstack(test_features2)

In [0]:
np.save("features/train_feat1.npy", train_feat1) 
np.save("features/train_feat2.npy", train_feat2)
np.save("features/train_labels.npy", train_labels)


In [0]:
np.save("features/test_feat1.npy", test_feat1) 
np.save("features/test_feat2.npy", test_feat2)
np.save("features/test_labels.npy", test_labels)


In [0]:
def package_features(feature_extractor, face_detector, save_features_folder="data/processed_2/packaged/features", FEATURES_PATH="data/processed_2/features", IMG_RT_PATH='data/images/'):
  seq = iaa.Sequential([iaa.MotionBlur(k = 20)])
  feat_path_list = []
  feat_path_list = []
  folder_list = os.listdir(FEATURES_PATH)
  for FOLDER in tqdm(folder_list):
      image_list_names = os.listdir(FEATURES_PATH + FOLDER)
      for NAME in image_list_names:
          feat_path_list.append(FOLDER+'/'+NAME)

  img_list = pd.DataFrame({"path": feat_path_list})
  iman = ImageManager(
      imgpaths = img_list,
      feature_extractor = feature_extractor,
      face_detector =  face_detector, 
      IMG_RTPATH = IMG_RT_PATH,
  )
  iman.balance_random_sample(100)
  paths = iman.get_sample_image_paths()
  iman.split(train_size = 0.8)
  path1 = np.array(img_list)[iman.sample_feat1]
  path2 = np.array(img_list)[iman.sample_feat2]
  path1, path2 = shuffle(path1, path2, random_state = 42)
  xtrain1 = path1[:int(len(path1)*0.8)]
  xtrain2 = path2[:int(len(path2)*0.8)]
  xtest1 = path1[int(len(path1)*0.8):]
  xtest2 = path2[int(len(path2)*0.8):]
  i = 0
  augment_mode = False
  method = "imgaug"
  training_features1 = []
  training_features2 = []
  p2, p1, train_labels = [], [], []
  path1 = xtrain1
  path2 = xtrain2
  for i in tqdm(range(len(path1))):
      training_features1.append(np.load(FEATURES_PATH + str(path1[i][0])))
      training_features2.append(np.load(FEATURES_PATH + str(path2[i][0])))
      if(path1[i][0].split('/')[0] ==path2[i][0].split('/')[0]):
          train_labels.append(1)
      else:
          train_labels.append(0)
  path1_t = xtest1
  path2_t = xtest2
  test_features1, test_features2 = [], []
  test_labels = []
  for i in tqdm(range(len(path1_t))):
      test_features1.append(np.load(FEATURES_PATH + path1_t[i][0]))
      test_features2.append(np.load(FEATURES_PATH + path2_t[i][0]))
      if(path1_t[i][0].split('/')[0] == path2_t[i][0].split('/')[0]):
          test_labels.append(1)
      else:
          test_labels.append(0)
  train_feat1 = np.vstack(training_features1)
  train_feat2 = np.vstack(training_features2)
  test_feat1 = np.vstack(test_features1)
  test_feat2 = np.vstack(test_features2)
  save_features(train_feat1, train_feat2, train_labels, test_feat1, test_feat2, test_labels, folder=save_features_folder)

def save_features(train_feat1, train_feat2, train_labels, test_feat1, test_feat2, test_labels, folder="features"):
  np.save(folder+"train_feat1.npy", train_feat1) 
  np.save(folder+"train_feat2.npy", train_feat2)
  np.save(folder+"train_labels.npy", train_labels)
  np.save(folder+"test_feat1.npy", test_feat1) 
  np.save(folder+"test_feat2.npy", test_feat2)
  np.save(folder+"test_labels.npy", test_labels)


In [18]:
package_features(feature_extractor, face_detector, save_features_folder="data/processed_2/packaged/features/", FEATURES_PATH="data/processed_2/features/", IMG_RT_PATH='data/images/')

100%|██████████| 74/74 [00:00<00:00, 851.41it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
100%|██████████| 2960/2960 [00:08<00:00, 363.45it/s]


In [0]:
package_features(feature_extractor, face_detector, save_features_folder="data/processed_2/packaged/motion_blur_features/", FEATURES_PATH="data/processed_2/motion_blur_features/", IMG_RT_PATH='data/images/')

100%|██████████| 74/74 [00:23<00:00,  3.12it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
  8%|▊         | 1004/11840 [08:37<1:18:00,  2.32it/s]